In [1]:
import numpy as np
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Read excel file with sheet name
dict_df = pd.read_excel('RFM_Dataset.xlsx', sheet_name= ["Orders", "Return","Segmentation"])


In [4]:
dict_df

{'Orders':       Row ID        Order ID Order Date  Ship Date       Ship Mode  \
 0        646  CA-2017-126221 2017-12-30 2018-01-05  Standard Class   
 1        907  CA-2017-143259 2017-12-30 2018-01-03  Standard Class   
 2        908  CA-2017-143259 2017-12-30 2018-01-03  Standard Class   
 3        909  CA-2017-143259 2017-12-30 2018-01-03  Standard Class   
 4       1297  CA-2017-115427 2017-12-30 2018-01-03  Standard Class   
 ...      ...             ...        ...        ...             ...   
 9989    1760  CA-2014-141817 2014-01-05 2014-01-12  Standard Class   
 9990     740  CA-2014-112326 2014-01-04 2014-01-08  Standard Class   
 9991     741  CA-2014-112326 2014-01-04 2014-01-08  Standard Class   
 9992     742  CA-2014-112326 2014-01-04 2014-01-08  Standard Class   
 9993    7981  CA-2014-103800 2014-01-03 2014-01-07  Standard Class   
 
      Customer ID      Channel  Postal Code       Product ID    Sales  \
 0       CC-12430  Home Office        47201  OFF-AP-10002457  2